In [0]:
import python_speech_features
import os, sys

import numpy as np
import matplotlib.pyplot as plt

# nvidia-smi --gpu-reset
import tensorflow as tf
from tensorflow import keras
import scipy.misc  # for image resizing

#import scipy.io.wavfile

# pip install soundfile
import soundfile
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
do_training = True

In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from google.cloud import storage
client = storage.Client("courseproject-239718")
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('courseproject')

In [0]:
!mkdir data

In [20]:
!gsutil -m cp -r gs://courseproject/data ./data

Copying gs://courseproject/data/non_speech_out/6_0.wav...
Copying gs://courseproject/data/non_speech_out/6_1.wav...
Copying gs://courseproject/data/non_speech_out/6_2.wav...
Copying gs://courseproject/data/non_speech_out/6_3.wav...
Copying gs://courseproject/data/non_speech_out/non_speech0.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1.wav...
Copying gs://courseproject/data/non_speech_out/non_speech10.wav...
Copying gs://courseproject/data/non_speech_out/non_speech100.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1000.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1001.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1002.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1003.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1004.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1005.wav...
Copying gs://courseproject/data/non_speech_out/non_speech1006.wav...
Copying gs://cours

In [0]:
sample_window_step = 0.01
def get_sample_features(samples, sample_rate):
    #sample_feat = python_speech_features.mfcc(samples, sample_rate, numcep=13, nfilt=26, appendEnergy=True)
    #sample_feat = python_speech_features.mfcc(samples, sample_rate, numcep=28, nfilt=56, appendEnergy=True)

    #sample_feat, e = python_speech_features.fbank(samples,samplerate=sample_rate,
    #      winlen=0.025,winstep=0.01,nfilt=26,nfft=512,
    #      lowfreq=0,highfreq=None,preemph=0.97, winfunc=lambda x:np.ones((x,)))

    features, energy = python_speech_features.fbank(samples, samplerate=sample_rate, 
                            winlen=0.025, winstep=sample_window_step, 
                            nfilt=32,nfft=512,
                            lowfreq=0,highfreq=None,preemph=0.25,
                            winfunc=lambda x:np.hamming( x ))
    return features, energy
# Convert a given (isolated word) WAV into a 'stamp' - using a helper function

def samples_to_stamp(samples, sample_rate):
    sample_feat, energy = get_sample_features(samples, sample_rate)
    
    data = np.log(sample_feat)
    
    # Now normalize each vertical slice so that the minimum energy is ==0
    data_mins = np.min(data, axis=1)
    data_min0 = data - data_mins[:, np.newaxis]
    
    # Force the data into the 'stamp size' as an image (implicit range normalization occurs)
    stamp = scipy.misc.imresize(data_min0, (64, 32), 'bilinear')
    stamp=np.expand_dims(stamp/ 255.0, -1)
    
    # https://github.com/scipy/scipy/issues/4458 :: The stamps are stored as uint8...
    return stamp

def wav_to_stamp(wav):
    samples, sample_rate = soundfile.read(wav) 
    return samples_to_stamp(samples, sample_rate)

In [0]:
speech_folder="data/data/speech_out"
no_speech_folder="data/data/non_speech_out"

In [0]:
import os
import random
# list file in folder
def list_files(folder):
    for r, d, f in os.walk(folder):
        for file in f:
            yield(os.path.join(r, file))
def count_files(folder):
    i=-1
    for i,file in enumerate(list_files(folder)):
        continue
    return i+1
def rand_gen_file(folder,n=1,seed=None):
    file_num=count_files(folder)
    if not seed:
        random.seed(seed)
    rand_num=random.sample(range(file_num), n)
    file_list=[]
    for i,file in enumerate(list_files(folder)):
        if i in rand_num:
            file_list.append(file)
        continue
    return file_list

In [0]:
speech_file_num=count_files(speech_folder)
no_speech_num=count_files(no_speech_folder)
speech_file_shuffle=rand_gen_file(speech_folder,speech_file_num,seed=823)
no_speech_file_shuffle=rand_gen_file(no_speech_folder,no_speech_num,seed=972)
train_speech_file=speech_file_shuffle[:int(0.8*speech_file_num)]
eval_speech_file=speech_file_shuffle[int(0.8*speech_file_num):]
train_no_speech_file=no_speech_file_shuffle[:int(0.8*no_speech_num)]
eval_no_speech_file=no_speech_file_shuffle[int(0.8*no_speech_num):]
train_files=train_speech_file+train_no_speech_file
eval_files=eval_speech_file+eval_no_speech_file

In [0]:
speech_label=[[1,0]]
no_speech_label=[[0,1]]
train_label=(speech_label*int(0.8*speech_file_num))+(no_speech_label*(int(0.8*no_speech_num)))
eval_label=(speech_label*(speech_file_num-int(0.8*speech_file_num)))+(no_speech_label*(no_speech_num-int(0.8*no_speech_num)))

In [0]:
def generat_single_example(file_list,label_list):
    for i,file in enumerate(file_list):
        yield (wav_to_stamp(file),label_list[i])

In [0]:
def make_dataset(files,label,num_epoch,batch_size,train,dataset_size):
    ds=tf.data.Dataset.from_generator(lambda:generat_single_example(files,label),(tf.float32,tf.int16),(tf.TensorShape([64,32,1]), tf.TensorShape([2])))
    if train:
        ds=ds.shuffle(dataset_size)
    ds=ds.repeat().batch(batch_size)
    return ds

In [10]:
train_dataset = make_dataset(train_files,train_label,20,20,True,int(0.8*speech_file_num)+int(0.8*no_speech_num))
eval_dataset=make_dataset(eval_files,eval_label,1,1,False,(speech_file_num-int(0.8*speech_file_num)+no_speech_num-int(0.8*no_speech_num)))

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [0]:
num_classes = 2 #with speech and no_speech

input_shape = (64, 32, 1)  # tf backend is channels_last

batch_size = 20
num_epochs = 20
checkpoint = ModelCheckpoint('./gdrive/My Drive/new_weights_{epoch:02d}_{val_acc:.2f}.hdf5', verbose=1, save_best_only=True, mode='auto')

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(16, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 28, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 10, 16)        6416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 5, 16)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1040)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                33312     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy']
             )



In [0]:
spe = (int(0.8*speech_file_num)+int(0.8*no_speech_num))// batch_size

In [0]:
model.fit(train_dataset, steps_per_epoch=spe, epochs=num_epochs, 
          validation_data=eval_dataset, validation_steps=(speech_file_num-int(0.8*speech_file_num)+no_speech_num-int(0.8*no_speech_num)), 
          verbose=1,callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


585/586 [============================>.] - ETA: 0s - loss: 0.3566 - acc: 0.8431
Epoch 00001: val_loss improved from inf to 0.21546, saving model to ./gdrive/My Drive/new_weights_01_0.92.hdf5
586/586 [==============================] - 188s 321ms/step - loss: 0.3562 - acc: 0.8433 - val_loss: 0.2155 - val_acc: 0.9165
Epoch 2/20
585/586 [============================>.] - ETA: 0s - loss: 0.2343 - acc: 0.9151
Epoch 00002: val_loss did not improve from 0.21546
586/586 [==============================] - 111s 189ms/step - loss: 0.2342 - acc: 0.9150 - val_loss: 0.2235 - val_acc: 0.9131
Epoch 3/20
585/586 [============================>.] - ETA: 0s - loss: 0.1912 - acc: 0.9332
Epoch 00003: val_loss improved from 0.21546 to 0.16954, saving model to ./gdrive/My Drive/new_weights_03_0.94.hdf5
586/586 [==============================] - 109s 186ms/step - loss: 0.1909 - acc: 0.9333 - val_loss: 0.1695 - val_acc: 0.9352
Epoch 4/20
101/586 [====>.........................] - ETA: 1:08 - loss: 0.1725 - acc: 

In [0]:
score = model.evaluate(eval_dataset, steps=(speech_file_num-int(0.8*speech_file_num)+no_speech_num-int(0.8*no_speech_num), verbose=1)
score